In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import math
from matplotlib import pyplot
from matplotlib.image import imread
import tensorflow as tf
from tensorflow import keras
import cv2
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image

from tensorflow.python.keras.utils import layer_utils
from tensorflow.python.keras.utils.data_utils import get_file
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.python.keras.utils.vis_utils import model_to_dot
#from tensorflow.keras.utils import plot_model
#from resnets_utils import *
from tensorflow.keras.initializers import glorot_uniform


from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

#allow GPU memory growth because TF 2.1 and CUDNN aren't getting along right now!
#config = tf.compat.v1.ConfigProto()
#config.gpu_options.allow_growth = True
#sess = tf.compat.v1.Session(config=config)

with open('/home/garrett/KagglesData/Butterflies/fgvc_fg_training.json','r') as anno_train:
    train = json.load(anno_train)
with open('/home/garrett/KagglesData/Butterflies/fgvc_fg_testing.json','r') as anno_test:
    test = json.load(anno_test)

#print(train)   


train.keys()
test.keys()
test_df = pd.DataFrame()
test_df = test_df.append(test['images'], ignore_index=True)
train_df = pd.DataFrame()
train_df = train_df.append(train['images'], ignore_index=True)
train_df_anno = pd.DataFrame()
train_df_anno = train_df_anno.append(train['annotations'], ignore_index=True)
train_df['category_id'] = train_df_anno['category_id']
test_df = test_df.append(test['images'], ignore_index=True)
test_df_anno = pd.DataFrame()

train_df.head()
foldertrain = '/home/garrett/KagglesData/Butterflies/data-images/training/images/'
foldertest = '/home/garrett/KagglesData/Butterflies/data-images/testing/images/'


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7011650025613100105
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3964141568
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10889746553803972141
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [11]:

#the images are all different sizes, so we will need to resize them. Size up to 600x600

#Takes dataframe and image directory, returns 600x600x3x(SIZE) ndarray of images.
#This is very slow, so to avoid looping over the dataframe more times than necessary we combine the function
# to get labels and the function to creat padded images into one.

def getLabelsAndPaddedImages(df, folder, isTrain):
    #slicesize = len(df.index)
    slicesize = 1000
    cat_labels = []
    images = []
    for i in range(slicesize):
        # define subplot
        # define filename
        filename = folder + df.at[i, 'file_name']
        # load image pixels. Images are rgb
        image = imread(filename)
        imresize = cv2.resize(image, (600,600),interpolation = cv2.INTER_AREA)
        #plot raw rgb pixel data
        #if i < 9:
        #    pyplot.subplot(330 + 1 + i)
        #    pyplot.imshow(imresize)
        # show the figure
        #pyplot.show()
        images.append(imresize)
        if(i%100 == 0):
            print("got image "),
            print(i)
        if isTrain == 1:
            labels = np.zeros(5419)
            cat_label = df.at[i,'category_id']
            labels[cat_label] = 1
            cat_labels.append(labels)
    if (isTrain == 1):
            #print(cat_labels)
            return (cat_labels, images)
    else: return images

In [12]:

train_labels_tot,train_img_tot = getLabelsAndPaddedImages(train_df, foldertrain, 1)
test_img = getLabelsAndPaddedImages(test_df, foldertest,0)

c = list(zip(train_labels_tot, train_img_tot))
np.random.shuffle(c)
train_labels_tot_shuf, train_img_tot_shuf = zip(*c)

train_labels = np.array(train_labels_tot_shuf[:int(len(train_labels_tot_shuf)/2)])
validation_labels = np.array(train_labels_tot_shuf[int(len(train_labels_tot_shuf)/2):])
train_img = np.array(train_img_tot_shuf[:int(len(train_img_tot_shuf)/2)])
validation_img = np.array(train_img_tot_shuf[int(len(train_img_tot_shuf)/2):])

print(len(train_img))


got image 
0
got image 
100
got image 
200
got image 
300
got image 
400
got image 
500
got image 
600
got image 
700
got image 
800
got image 
900
got image 
0
got image 
100
got image 
200
got image 
300
got image 
400
got image 
500
got image 
600
got image 
700
got image 
800
got image 
900
500


In [13]:
#Now we build the basic Convnet. Use Tensorflow + Keras for this on my RTX gpu.
#Let's make ResNet from scratch in Keras. First, define the basic residual module.

def identity_block(X, f, filters, stage, block):
    
    # defining name basis
    cNN_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_forlater = X
    
    # First CNN, batchnorm, and RELU. Use Glorot uniform init for the CNN layers and ELU activation.
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = cNN_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('elu')(X)

    # Second CNN, batchnorm, and RELU.
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = cNN_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('elu')(X)

    # Third CNN and batchnorm
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = cNN_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add original X value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_forlater])
    X = Activation('elu')(X)
    
    
    return X

In [14]:
def convolutional_block(X, f, filters, stage, block, s = 2):

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_forlater = X

    # First CNN, batchnorm, and RELU. Use Glorot uniform init for the CNN layers and ELU activation.

    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('elu')(X)

    # Second CNN, batchnorm, and RELU. Use Glorot uniform init for the CNN layers and ELU activation.
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('elu')(X)


    # Third CNN, batchnorm, and RELU. Use Glorot uniform init for the CNN layers and ELU activation.
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    X_forlater = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_forlater)
    X_forlater = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_forlater)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_forlater])
    X = Activation('elu')(X)
    
    return X

In [ ]:
def ResNet50(input_shape=(600, 600, 3), classes=6):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('elu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:


X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

In [ ]:
model.fit(X_train, Y_train, epochs = 25, batch_size = 32)

In [ ]:
epochs = 1
batch_size = 1
history = model.fit(train_img,train_labels, batch_size=batch_size,epochs = epochs, validation_data = (validation_img,validation_labels))

print('\n# Generate predictions for test set')
predictions = model.predict(np.array(test_img))
print('predictions shape:', predictions.shape)
predvals = tf.math.argmax(predictions, axis = 1)
print(predvals)

In [ ]:



# Define a sequential model
input_shape = (600,600,3)
num_classes= 5419
model = Sequential()
# add first convolutional layer
model.add(Conv2D(32, kernel_size=(7, 7), activation='elu', input_shape=input_shape))
# add second convolutional layer
model.add(Conv2D(32, (5, 5), activation='elu'))
# add one max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))
# add one dropout layer
model.add(Dropout(0.25))
# add flatten layer
model.add(Flatten())
# add dense layer
model.add(Dense(16, activation='relu'))
# add another dropout layer
model.add(Dropout(0.5))
# add dense layer
model.add(Dense(num_classes, activation='softmax'))
# complile the model and view its architecture
model.compile(loss=keras.losses.categorical_crossentropy,  optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
print(model.summary())
